A classe chamada GeneralizedSidelobeCanceller foi reformulada. Agora são duas classes: DaSParallel e GSC.

In [12]:
from random import random
from IPython.display import Audio
import numpy as np
import librosa
import sys
import os
sys.path.append("/home/dimi/Programming/IC2019/DOA/Python/")
from DaSParallelClass import DaSParallel
from GSCClass import GSC
from SimularArrayMicrofonesClass import SimularArrayMicrofones
from GeneralizedSidelobeCancellerClass import GeneralizedSidelobeCanceller

In [14]:
# COORDENADAS 
qtdMics = 4
coordenadasMics = []
for i in range(qtdMics):
    coordenadasMics.append([random()/10, random()/10, random()/10])
coordenadasMics = np.array(coordenadasMics)

# SEMPRE COLOCAR O MICROFNE DE REFERENCIA NA ORIGEM
for i, coordenadaAtual in enumerate(coordenadasMics):
    coordenadasMics[i] = coordenadaAtual - coordenadasMics[0]

# SIMULANDO O ARRAY DE MICROFONES ALEATORIAMENTE
energiaRuido = 0.05
objSimularArrayMics = SimularArrayMicrofones("/home/dimi/Programming/IC2019/DOA/Gravacoes/Simulacoes/Gravacao3/sinalPuroMono6Segundos.wav", energiaRuido, normalizarEnergia=True, coordenadasMics=coordenadasMics)
arraySinaisMics, delaysVerdadeiros, freqAmostragem = objSimularArrayMics.obterResultado()

In [17]:
objDaS = DaSParallel(arraySinaisMics, qtdNucleos=1)
print("Tempo de Processamento do DaS novo (s):", objDaS.tempoProcessamento)

objGSC = GSC(arraySinaisMics, objDaS.sinalDaS)
print("Tempo de Processamento da parte adaptativa do GSC novo (s):", objGSC.tempoProcessamento)
print("Tempo de Processamento total do GSC novo (s):", objGSC.tempoProcessamento+objDaS.tempoProcessamento)

Tempo de Processamento do DaS novo (s): 0.13226008415222168
Tempo de Processamento da parte adaptativa do GSC novo (s): 1.1533443927764893
Tempo de Processamento total do GSC novo (s): 1.285604476928711


In [21]:
print("SEM DaS")
print("Energia:", np.var(np.sum(arraySinaisMics, axis=0)/len(arraySinaisMics)))
Audio(data=np.sum(arraySinaisMics, axis=0)/len(arraySinaisMics), rate=freqAmostragem)

SEM DaS
Energia: 0.9236493831749643


In [22]:
print("COM DaS")
print("Energia:", np.var(objDaS.sinalDaS))
Audio(data=objDaS.sinalDaS, rate=freqAmostragem)

COM DaS
Energia: 1.012760665468076


In [23]:
print("COM GSC")
print("Energia:", np.var(objGSC.sinalGSC))
Audio(data=objGSC.sinalGSC, rate=freqAmostragem)

COM GSC
Energia: 0.7537281100578704


## Comparando o tempo de processamento com quantidades de núcleos diferentes

### Array de 4 mics

In [24]:
arrayTemposUmNucleo     = []
arrayTemposTodosNucleos = []

# PARA CADA AUDIO DO SESA DE TESTE
audios = os.listdir("/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/test/")
for i, audioAtual in enumerate(audios):
    
    # SIMULO UM ARRAY DE MICS
    caminho = "/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/test/" + audioAtual
    objSimularArrayMics = SimularArrayMicrofones(caminho, energiaRuido=0.1)
    arraySinaisMics, __, __ = objSimularArrayMics.obterResultado()
    
    # APLICO O DAS
    objDaSUmNucleo     = DaSParallel(arraySinaisMics, qtdNucleos=1)
    objDaSTodosNucleos = DaSParallel(arraySinaisMics, qtdNucleos=-1)
    arrayTemposUmNucleo.append(objDaSUmNucleo.tempoProcessamento)
    arrayTemposTodosNucleos.append(objDaSTodosNucleos.tempoProcessamento)
    
    print("Progresso: " + str(100*((i+1)/len(audios))) + "%")

print("ARRAY DE 4 MICS:")
print("\n\nTempo com um único núcleo: ", np.mean(arrayTemposUmNucleo), "+-", np.std(arrayTemposUmNucleo))
print("Tempo com todos os núcleos:", np.mean(arrayTemposTodosNucleos), "+-", np.std(arrayTemposTodosNucleos))

Progresso: 0.9523809523809524%
Progresso: 1.9047619047619049%
Progresso: 2.857142857142857%
Progresso: 3.8095238095238098%
Progresso: 4.761904761904762%
Progresso: 5.714285714285714%
Progresso: 6.666666666666667%
Progresso: 7.6190476190476195%
Progresso: 8.571428571428571%
Progresso: 9.523809523809524%
Progresso: 10.476190476190476%
Progresso: 11.428571428571429%
Progresso: 12.380952380952381%
Progresso: 13.333333333333334%
Progresso: 14.285714285714285%
Progresso: 15.238095238095239%
Progresso: 16.19047619047619%
Progresso: 17.142857142857142%
Progresso: 18.095238095238095%
Progresso: 19.047619047619047%
Progresso: 20.0%
Progresso: 20.952380952380953%
Progresso: 21.904761904761905%
Progresso: 22.857142857142858%
Progresso: 23.809523809523807%
Progresso: 24.761904761904763%
Progresso: 25.71428571428571%
Progresso: 26.666666666666668%
Progresso: 27.61904761904762%
Progresso: 28.57142857142857%
Progresso: 29.523809523809526%
Progresso: 30.476190476190478%
Progresso: 31.428571428571427%
P